In [118]:
import pandas as pd
import numpy as np
import scipy as sp
import seaborn as sn
import folium
import math
import json
from branca.colormap import LinearColormap


In [119]:
intercountry_df = pd.read_csv('../Data_sinan/intercountry_tone_count_table_with_EVENTCODE',\
                              dtype={'EventRootCode': np.str, 'EventCode': np.str} \
                             )
world_borders = json.load(open('../Data/world_borders.topojson.json'))
fips_codes = pd.read_csv('../Data/fipscountries.csv')
fips_codes = fips_codes[['fips','country']]

In [120]:
intercountry_df = intercountry_df[pd.notnull(intercountry_df['LocEvent'])]
intercountry_df = (intercountry_df[(intercountry_df["EventRootCode"]!='--') & (intercountry_df["EventCode"]!='--')])

In [121]:
intercountry_df = fips_codes.merge(intercountry_df, left_on='fips', right_on='LocEvent', how='inner')

#On the topojson data, the country names don't match
intercountry_df.loc[intercountry_df['country']=='United States', 'country'] = 'United States of America'
intercountry_df.loc[intercountry_df['country']=='Serbia', 'country'] = 'Republic of Serbia'

In [122]:
intercountry_nb_mentions_df = intercountry_df[['country', 'EventCode', 'NumberOfInterMentions']]
intercountry_nb_mentions_df = intercountry_nb_mentions_df.groupby(['country', 'EventCode']).sum()
intercountry_nb_mentions_df = intercountry_nb_mentions_df.reset_index(level=[0,1])
intercountry_nb_mentions_df.head()

,country,EventCode,NumberOfInterMentions
0,Afghanistan,010,715328
1,Afghanistan,011,14931
2,Afghanistan,012,121362
3,Afghanistan,013,156894
4,Afghanistan,014,46137


In [123]:
#GET THE MOST DISCUSSED EVENT OCCURING IN A COUNTRY
s = intercountry_nb_mentions_df.groupby('country').apply(lambda x: x.EventCode[x.NumberOfInterMentions.idxmax()])
a = pd.DataFrame({'country':s.index, 'EventCode':s.values})
a.head()



,country,EventCode
0,Afghanistan,190
1,Albania,040
2,Algeria,042
3,American Samoa,110
4,Andorra,010


In [124]:
a['EventCode'].value_counts()

042    92
010    57
040    39
190    17
043    12
020     6
051     5
173     3
061     1
110     1
036     1
046     1
Name: EventCode, dtype: int64

In [125]:
#COUNTRIES KNOWN BY THE WAR ON THE MEDIA
war_countries = a[a['EventCode'].str.contains('19')]

In [126]:
happiness_df = pd.read_excel('../Data_sinan/WHR2018Chapter2OnlineData.xls')
happiness_df = happiness_df[happiness_df['year']==2016][['country','Confidence in national government', 'Democratic Quality']]
happiness_df.head()

,country,Confidence in national government,Democratic Quality
8,Afghanistan,0.324990,-1.917693
18,Albania,0.400910,0.208456
24,Algeria,NaN,-1.008262
40,Argentina,0.419562,0.381370
52,Armenia,0.184713,-0.609075


In [128]:
war_countries = war_countries.merge(happiness_df, on='country', how='inner')

In [130]:
war_countries

,country,EventCode,Confidence in national government,Democratic Quality
0,Afghanistan,190,0.324990,-1.917693
1,Cameroon,190,0.621221,-0.988150
2,Central African Republic,190,0.748755,-1.433271
3,Iraq,190,0.418604,-1.643929
4,Israel,190,0.438133,-0.027976
5,Libya,190,NaN,-1.790413
6,Mali,190,0.524199,-0.864826
7,Niger,190,0.572468,-0.711488
8,Somalia,190,0.700682,-2.081906
9,South Sudan,190,0.456288,-2.044661
